**Group Details:**



1.   Janusz Feigel
2.   Bhavana Malla
3.   Brinda Rao  







In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Introduction to Deep Learning')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/Introduction to Deep Learning'

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datasets import MNISTDataset
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,Model
from tensorflow.keras import layers
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Input,Dropout,MaxPool2D, AvgPool2D, GlobalAveragePooling2D,concatenate, BatchNormalization,Add, ReLU,GlobalAvgPool2D
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import time
from datetime import datetime

In [ ]:
logdir = os.path.join("logs", "linear" + str(datetime.now()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height = x_train.shape[1]
img_width = x_train.shape[2]
channels = x_train.shape[3]
num_classes = max(y_train) + 1

epochs = 10
batch_size = 256
train_size = x_train.shape[0]
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(train_size).repeat(epochs).batch(batch_size)         

model = tf.keras.Sequential(
    [layers.Rescaling(1./255, input_shape=(img_height, img_width, channels)),
     layers.Conv2D(16, 5, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(32, 5, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(64, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Flatten(),
     layers.Dense(128, activation='relu'),
     layers.Dense(num_classes)     
    ]
)

model_large = tf.keras.Sequential(
    [layers.Rescaling(1./255, input_shape=(img_height, img_width, channels)),
     layers.Conv2D(16, 5, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(32, 5, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(64, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(64, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(64, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Conv2D(64, 3, padding='same', activation='relu'),
     layers.MaxPooling2D(padding='same'),
     layers.Flatten(),
     layers.Dense(512, activation='relu'),
     layers.Dense(256, activation='relu'),
     layers.Dense(128, activation='relu'),
     layers.Dense(num_classes)     
    ]
)

optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = keras.metrics.SparseCategoricalAccuracy()

## **with tf.function and small model**

In [ ]:
#with tf.function and small model
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    accuracy.update_state(y, logits)
    return loss_value

i = 0
epoch = 1
times_tf_function = []
for step, (img_batch, lbl_batch) in enumerate(train_data):
    i += batch_size
    t1 = time.time()
    loss_value = train_step(img_batch, lbl_batch)
    t2 = time.time()
    times_tf_function.append(t2-t1)

    
    if i / train_size > 1:
        train_acc = accuracy.result().numpy()
        accuracy.update_state(y_test, model(x_test, training=False))
        test_acc = accuracy.result().numpy()
        print("Epoch {} Train Accuracy: {} Test Accuracy: {} Train Loss: {}".format(epoch, train_acc, test_acc, float(loss_value)))

        with train_writer.as_default():
            tf.summary.scalar("accuracy", train_acc, step=epoch)
            tf.summary.scalar("loss", float(loss_value), step=epoch)

        with test_writer.as_default():
            tf.summary.scalar("accuracy", test_acc, step=epoch)
        i -= train_size
        epoch += 1

Epoch 1 Train Accuracy: 0.37517938017845154 Test Accuracy: 0.39067068696022034 Train Loss: 1.5447204113006592
Epoch 2 Train Accuracy: 0.4451387822628021 Test Accuracy: 0.4541366994380951 Train Loss: 1.266302227973938
Epoch 3 Train Accuracy: 0.4868541657924652 Test Accuracy: 0.49237844347953796 Train Loss: 1.1267685890197754
Epoch 4 Train Accuracy: 0.5164341330528259 Test Accuracy: 0.5207375884056091 Train Loss: 1.0072190761566162
Epoch 5 Train Accuracy: 0.5395123362541199 Test Accuracy: 0.5425074696540833 Train Loss: 1.0147024393081665
Epoch 6 Train Accuracy: 0.5581461191177368 Test Accuracy: 0.5605168342590332 Train Loss: 1.0540354251861572
Epoch 7 Train Accuracy: 0.5740770697593689 Test Accuracy: 0.5757648348808289 Train Loss: 0.9022608995437622
Epoch 8 Train Accuracy: 0.5876740217208862 Test Accuracy: 0.5894011855125427 Train Loss: 0.7587568759918213
Epoch 9 Train Accuracy: 0.6002626419067383 Test Accuracy: 0.6017261147499084 Train Loss: 0.7776539921760559
Epoch 10 Train Accuracy: 0

## **without tf.function and small model**

In [ ]:
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    accuracy.update_state(y, logits)
    return loss_value

times_without_tf_function = []
i = 0
epoch = 1
for step, (img_batch, lbl_batch) in enumerate(train_data):
    i += batch_size
    t1 = time.time()
    loss_value = train_step(img_batch, lbl_batch)
    t2 = time.time()
    times_without_tf_function.append(t2-t1)

    
    if i / train_size > 1:
        train_acc = accuracy.result().numpy()
        accuracy.update_state(y_test, model(x_test, training=False))
        test_acc = accuracy.result().numpy()
        print("Epoch {} Train Accuracy: {} Test Accuracy: {} Train Loss: {}".format(epoch, train_acc, test_acc, float(loss_value)))

        with train_writer.as_default():
            tf.summary.scalar("accuracy", train_acc, step=epoch)
            tf.summary.scalar("loss", float(loss_value), step=epoch)
            
        with test_writer.as_default():
            tf.summary.scalar("accuracy", test_acc, step=epoch)
        i -= train_size
        epoch += 1

Epoch 1 Train Accuracy: 0.6198337078094482 Test Accuracy: 0.620635986328125 Train Loss: 0.6965063214302063
Epoch 2 Train Accuracy: 0.6294909715652466 Test Accuracy: 0.6303298473358154 Train Loss: 0.6904239058494568
Epoch 3 Train Accuracy: 0.6386036276817322 Test Accuracy: 0.6392061114311218 Train Loss: 0.6249908208847046
Epoch 4 Train Accuracy: 0.6469732522964478 Test Accuracy: 0.6475889086723328 Train Loss: 0.6162972450256348
Epoch 5 Train Accuracy: 0.6548692584037781 Test Accuracy: 0.6552839875221252 Train Loss: 0.6784425377845764
Epoch 6 Train Accuracy: 0.6620082259178162 Test Accuracy: 0.6623935699462891 Train Loss: 0.6900976300239563
Epoch 7 Train Accuracy: 0.6690909266471863 Test Accuracy: 0.6693556904792786 Train Loss: 0.6182703971862793
Epoch 8 Train Accuracy: 0.6756565570831299 Test Accuracy: 0.6758291721343994 Train Loss: 0.6098261475563049
Epoch 9 Train Accuracy: 0.6818665862083435 Test Accuracy: 0.6820537447929382 Train Loss: 0.6085209250450134
Epoch 10 Train Accuracy: 0.68

## **with tf.function and large model**

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model_large(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model_large.trainable_weights)
    optimizer.apply_gradients(zip(grads, model_large.trainable_weights))
    accuracy.update_state(y, logits)
    return loss_value

i = 0
epoch = 1
times_tf_function_large = []
for step, (img_batch, lbl_batch) in enumerate(train_data):
    i += batch_size
    t1 = time.time()
    loss_value = train_step(img_batch, lbl_batch)
    t2 = time.time()
    times_tf_function_large.append(t2-t1)

    
    if i / train_size > 1:
        train_acc = accuracy.result().numpy()
        accuracy.update_state(y_test, model_large(x_test, training=False))
        test_acc = accuracy.result().numpy()
        print("Epoch {} Train Accuracy: {} Test Accuracy: {} Train Loss: {}".format(epoch, train_acc, test_acc, float(loss_value)))

        with train_writer.as_default():
            tf.summary.scalar("accuracy", train_acc, step=epoch)
            tf.summary.scalar("loss", float(loss_value), step=epoch)

        with test_writer.as_default():
            tf.summary.scalar("accuracy", test_acc, step=epoch)
        i -= train_size
        epoch += 1

Epoch 1 Train Accuracy: 0.578320324420929 Test Accuracy: 0.5731789469718933 Train Loss: 1.8740954399108887
Epoch 2 Train Accuracy: 0.554577112197876 Test Accuracy: 0.5516458749771118 Train Loss: 1.646367073059082
Epoch 3 Train Accuracy: 0.54250168800354 Test Accuracy: 0.5412863492965698 Train Loss: 1.3897863626480103
Epoch 4 Train Accuracy: 0.5372925996780396 Test Accuracy: 0.5367177724838257 Train Loss: 1.3687468767166138
Epoch 5 Train Accuracy: 0.535271942615509 Test Accuracy: 0.5350567698478699 Train Loss: 1.3128693103790283
Epoch 6 Train Accuracy: 0.5350314378738403 Test Accuracy: 0.5350092649459839 Train Loss: 1.1583517789840698
Epoch 7 Train Accuracy: 0.5359737873077393 Test Accuracy: 0.5361171364784241 Train Loss: 1.1415005922317505
Epoch 8 Train Accuracy: 0.5377365946769714 Test Accuracy: 0.5378825664520264 Train Loss: 1.1165262460708618
Epoch 9 Train Accuracy: 0.5399646759033203 Test Accuracy: 0.5401948094367981 Train Loss: 1.1555877923965454
Epoch 10 Train Accuracy: 0.5426419

## **without tf.function and large model**

In [ ]:
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model_large(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model_large.trainable_weights)
    optimizer.apply_gradients(zip(grads, model_large.trainable_weights))
    accuracy.update_state(y, logits)
    return loss_value

times_without_tf_function_large = []
i = 0
epoch = 1
for step, (img_batch, lbl_batch) in enumerate(train_data):
    i += batch_size
    t1 = time.time()
    loss_value = train_step(img_batch, lbl_batch)
    t2 = time.time()
    times_without_tf_function_large.append(t2-t1)

    
    if i / train_size > 1:
        train_acc = accuracy.result().numpy()
        accuracy.update_state(y_test, model_large(x_test, training=False))
        test_acc = accuracy.result().numpy()
        print("Epoch {} Train Accuracy: {} Test Accuracy: {} Train Loss: {}".format(epoch, train_acc, test_acc, float(loss_value)))

        with train_writer.as_default():
            tf.summary.scalar("accuracy", train_acc, step=epoch)
            tf.summary.scalar("loss", float(loss_value), step=epoch)
            
        with test_writer.as_default():
            tf.summary.scalar("accuracy", test_acc, step=epoch)
        i -= train_size
        epoch += 1

Epoch 1 Train Accuracy: 0.545363187789917 Test Accuracy: 0.5454881191253662 Train Loss: 1.0921859741210938
Epoch 2 Train Accuracy: 0.5484636425971985 Test Accuracy: 0.5486729741096497 Train Loss: 0.8611825108528137
Epoch 3 Train Accuracy: 0.5516840815544128 Test Accuracy: 0.551855206489563 Train Loss: 1.0415751934051514
Epoch 4 Train Accuracy: 0.5549912452697754 Test Accuracy: 0.5551114082336426 Train Loss: 1.0378799438476562
Epoch 5 Train Accuracy: 0.5585325360298157 Test Accuracy: 0.5586409568786621 Train Loss: 0.8624688982963562
Epoch 6 Train Accuracy: 0.5621657967567444 Test Accuracy: 0.5623538494110107 Train Loss: 0.9058047533035278
Epoch 7 Train Accuracy: 0.5660796761512756 Test Accuracy: 0.5662655830383301 Train Loss: 0.8777292370796204
Epoch 8 Train Accuracy: 0.569894015789032 Test Accuracy: 0.570025622844696 Train Loss: 0.8953149318695068
Epoch 9 Train Accuracy: 0.5736650228500366 Test Accuracy: 0.5737859010696411 Train Loss: 0.9410263299942017
Epoch 10 Train Accuracy: 0.57744

## **Comaprison b/w small and large models with and without tf.function decorator**

In [ ]:
print("With tf.function and small model a train step takes {}s".format(np.mean(times_tf_function)))
print("Without tf.function and small model a train step takes {}s".format(np.mean(times_without_tf_function)))
print("With tf.function and large model a train step takes {}s".format(np.mean(times_tf_function_large)))
print("Without tf.function and large model a train step takes {}s".format(np.mean(times_without_tf_function_large)))

With tf.function and small model a train step takes 0.013421702775398808s
Without tf.function and small model a train step takes 0.026557985079325087s
With tf.function and large model a train step takes 0.018656457092769314s
Without tf.function and large model a train step takes 0.033510945592468325s


## **DenseNet**

In [ ]:
from keras.datasets import cifar10
def load_dataset_cifar():
  # load dataset
  (trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
  #trainY = to_categorical(trainY)
  #testY = to_categorical(testY)
  return trainX, trainY, testX, testY


def prepare_dataset(trainX, trainY, testX, testY):

# Prepare the training dataset.
  train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainY))
  train_dataset = train_dataset.shuffle(buffer_size=50000).batch(128)

#Prepare the test dataset
  test_dataset =  tf.data.Dataset.from_tensor_slices((testX, testY))
  test_dataset = test_dataset.batch(128)

  return train_dataset,test_dataset

def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, kernel_initializer='he_normal', strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = Dropout(0.2)(y)
            y = bn_rl_conv(y, filters, 3)
            y = Dropout(0.2)(y)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = Dropout(0.2)(x)
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAvgPool2D()(d)
    flat =Flatten()(x)
    output = Dense(n_classes, activation = 'softmax')(flat)
    
    model = Model(input, output)
    return model

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    opt.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value
    
@tf.function
def test_step(x, y):
    test_logits = model(x, training=False)
    test_acc_metric.update_state(y, test_logits)

In [ ]:

input_shape = 32, 32, 3
n_classes = 10

# load dataset
trainX, trainY, testX, testY = load_dataset_cifar()
#prepare dataset
train_dataset,test_dataset = prepare_dataset(trainX, trainY, testX, testY)
# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)

trainX = datagen.fit(trainX)

# define model
model = densenet(input_shape,n_classes)
# compile model
opt = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy()

In [ ]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_228 (Conv2D)            (None, 16, 16, 64)   9472        ['input_11[0][0]']               
                                                                                                  
 max_pooling2d_28 (MaxPooling2D  (None, 8, 8, 64)    0           ['conv2d_228[0][0]']             
 )                                                                                                
                                                                                                  
 batch_normalization_165 (Batch  (None, 8, 8, 64)    256         ['max_pooling2d_28[0][0]'] 

In [ ]:
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot
import graphviz

SVG(model_to_dot(
    model, show_shapes=True, show_layer_names=True, rankdir='TB',
    expand_nested=False, dpi=60, subgraph=False
).create(prog='dot',format='svg'))

In [ ]:
# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
test_acc_metric = keras.metrics.SparseCategoricalAccuracy()

## **growth rate-24 without dropout and data augmentation--- Training acc: 0.9783 Test acc: 0.7023**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Validation acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 2.4178
Training loss (for one batch) at step 200: 1.2649
Training acc over epoch: 0.5004

Start of epoch 1
Training loss (for one batch) at step 0: 1.0503
Training loss (for one batch) at step 200: 1.2342
Training acc over epoch: 0.6260

Start of epoch 2
Training loss (for one batch) at step 0: 1.0458
Training loss (for one batch) at step 200: 1.0184
Training acc over epoch: 0.6885

Start of epoch 3
Training loss (for one batch) at step 0: 0.8029
Training loss (for one batch) at step 200: 0.8084
Training acc over epoch: 0.7274

Start of epoch 4
Training loss (for one batch) at step 0: 0.6412
Training loss (for one batch) at step 200: 0.6222
Training acc over epoch: 0.7567

Start of epoch 5
Training loss (for one batch) at step 0: 0.5394
Training loss (for one batch) at step 200: 0.5788
Training acc over epoch: 0.7813

Start of epoch 6
Training loss (for one batch) at step 0: 0.3362
Training loss (for one batch) at step 200: 0.4

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/func

## **growth rate 32 without dropout and data augmentation --Training acc: 0.9498 Test acc: 0.7779**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Validation acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 0.2480
Training loss (for one batch) at step 200: 0.2737
Training acc over epoch: 0.9145

Start of epoch 1
Training loss (for one batch) at step 0: 0.1943
Training loss (for one batch) at step 200: 0.2631
Training acc over epoch: 0.9176

Start of epoch 2
Training loss (for one batch) at step 0: 0.1921
Training loss (for one batch) at step 200: 0.2268
Training acc over epoch: 0.9194

Start of epoch 3
Training loss (for one batch) at step 0: 0.1605
Training loss (for one batch) at step 200: 0.1699
Training acc over epoch: 0.9193

Start of epoch 4
Training loss (for one batch) at step 0: 0.3000
Training loss (for one batch) at step 200: 0.1382
Training acc over epoch: 0.9204

Start of epoch 5
Training loss (for one batch) at step 0: 0.1875
Training loss (for one batch) at step 200: 0.2878
Training acc over epoch: 0.9227

Start of epoch 6
Training loss (for one batch) at step 0: 0.1889
Training loss (for one batch) at step 200: 0.2

## **growth rate-32, dropout-0.2 and with data augmentation--Training acc: 0.915 Test acc: 0.8022**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Validation acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 2.4580
Training loss (for one batch) at step 200: 1.3215
Training acc over epoch: 0.4541

Start of epoch 1
Training loss (for one batch) at step 0: 1.2222
Training loss (for one batch) at step 200: 1.1570
Training acc over epoch: 0.5846

Start of epoch 2
Training loss (for one batch) at step 0: 0.9636
Training loss (for one batch) at step 200: 1.0707
Training acc over epoch: 0.6476

Start of epoch 3
Training loss (for one batch) at step 0: 0.9656
Training loss (for one batch) at step 200: 0.8065
Training acc over epoch: 0.6809

Start of epoch 4
Training loss (for one batch) at step 0: 1.0205
Training loss (for one batch) at step 200: 0.8854
Training acc over epoch: 0.7071

Start of epoch 5
Training loss (for one batch) at step 0: 0.7880
Training loss (for one batch) at step 200: 0.7674
Training acc over epoch: 0.7269

Start of epoch 6
Training loss (for one batch) at step 0: 0.7082
Training loss (for one batch) at step 200: 0.5

## **growth rate-32 without batch and bottleneck layer with drop out and data augmentation--Training acc: 0.7846 Test acc: 0.7060**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Test acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 115.0205
Training loss (for one batch) at step 200: 1.9857
Training acc over epoch: 0.2551

Start of epoch 1
Training loss (for one batch) at step 0: 1.9081
Training loss (for one batch) at step 200: 1.5382
Training acc over epoch: 0.3710

Start of epoch 2
Training loss (for one batch) at step 0: 1.7053
Training loss (for one batch) at step 200: 1.6782
Training acc over epoch: 0.4184

Start of epoch 3
Training loss (for one batch) at step 0: 1.5528
Training loss (for one batch) at step 200: 1.5269
Training acc over epoch: 0.4466

Start of epoch 4
Training loss (for one batch) at step 0: 1.3650
Training loss (for one batch) at step 200: 1.5948
Training acc over epoch: 0.4711

Start of epoch 5
Training loss (for one batch) at step 0: 1.2683
Training loss (for one batch) at step 200: 1.3915
Training acc over epoch: 0.4913

Start of epoch 6
Training loss (for one batch) at step 0: 1.1629
Training loss (for one batch) at step 200: 1

## **growth rate-32 without batch and bottleneck layer with dropout-0.2 and data augmentation for dense block [3,6,12]--Training acc: 0.8001 Test acc: 0.7005**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Test acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 19.2920
Training loss (for one batch) at step 200: 1.7106
Training acc over epoch: 0.3333

Start of epoch 1
Training loss (for one batch) at step 0: 1.5611
Training loss (for one batch) at step 200: 1.4488
Training acc over epoch: 0.4618

Start of epoch 2
Training loss (for one batch) at step 0: 1.3985
Training loss (for one batch) at step 200: 1.4555
Training acc over epoch: 0.5106

Start of epoch 3
Training loss (for one batch) at step 0: 1.3388
Training loss (for one batch) at step 200: 1.1910
Training acc over epoch: 0.5459

Start of epoch 4
Training loss (for one batch) at step 0: 1.0767
Training loss (for one batch) at step 200: 1.1924
Training acc over epoch: 0.5674

Start of epoch 5
Training loss (for one batch) at step 0: 1.1860
Training loss (for one batch) at step 200: 1.0173
Training acc over epoch: 0.5873

Start of epoch 6
Training loss (for one batch) at step 0: 1.0267
Training loss (for one batch) at step 200: 1.

## **growth rate-32 with batch and bottleneck layer with drop out and data augmention dense layer [3,6,12]**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Test acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 2.4178
Training loss (for one batch) at step 200: 1.5706
Training acc over epoch: 0.4368

Start of epoch 1
Training loss (for one batch) at step 0: 1.1944
Training loss (for one batch) at step 200: 1.2739
Training acc over epoch: 0.5871

Start of epoch 2
Training loss (for one batch) at step 0: 0.8971
Training loss (for one batch) at step 200: 1.1137
Training acc over epoch: 0.6448

Start of epoch 3
Training loss (for one batch) at step 0: 0.8804
Training loss (for one batch) at step 200: 0.9684
Training acc over epoch: 0.6861

Start of epoch 4
Training loss (for one batch) at step 0: 0.8766
Training loss (for one batch) at step 200: 0.7041
Training acc over epoch: 0.7088

Start of epoch 5
Training loss (for one batch) at step 0: 0.6389
Training loss (for one batch) at step 200: 0.7270
Training acc over epoch: 0.7278

Start of epoch 6
Training loss (for one batch) at step 0: 0.6069
Training loss (for one batch) at step 200: 0.9

## **growth rate-32 with batch and bottleneck layer with drop out and data augmention dense layer DenseNet-121 [6,12,24,16]**

In [ ]:
import time


epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        loss_value = train_step(x_batch_train, y_batch_train)
         # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in test_dataset:
        test_step(x_batch_val, y_batch_val)

    test_acc = test_acc_metric.result()
    test_acc_metric.reset_states()
print("Test acc: %.4f" % (float(test_acc),))
print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 2.4802
Training loss (for one batch) at step 200: 1.6307
Training acc over epoch: 0.4030

Start of epoch 1
Training loss (for one batch) at step 0: 1.2116
Training loss (for one batch) at step 200: 1.4177
Training acc over epoch: 0.5611

Start of epoch 2
Training loss (for one batch) at step 0: 1.2640
Training loss (for one batch) at step 200: 0.9875
Training acc over epoch: 0.6333

Start of epoch 3
Training loss (for one batch) at step 0: 0.9368
Training loss (for one batch) at step 200: 0.9282
Training acc over epoch: 0.6798

Start of epoch 4
Training loss (for one batch) at step 0: 0.9436
Training loss (for one batch) at step 200: 0.7253
Training acc over epoch: 0.7141

Start of epoch 5
Training loss (for one batch) at step 0: 0.7796
Training loss (for one batch) at step 200: 0.8034
Training acc over epoch: 0.7385

Start of epoch 6
Training loss (for one batch) at step 0: 0.6545
Training loss (for one batch) at step 200: 0.5